# Методы первой необходмости

In [2]:
import os
os.environ['HF_HOME'] = 'D:\\HuggingFace'
os.environ['TRANSFORMERS_CACHE'] = os.environ['HF_HOME']
os.environ['HUGGINGFACE_HUB_CACHE'] = os.environ['HF_HOME'] 

In [13]:
from warnings import filterwarnings
filterwarnings('ignore')


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from transformers import pipeline
import re

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

sns.set_style('darkgrid')
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Предобработка текстов в pdf и создание датасета

In [11]:
import PyPDF2

In [12]:
clean_data = pd.DataFrame({"Informatics": [], "Art": [], "Space": [], "Sport": [], "Chemical": []})

In [166]:
import pathlib

#books = pathlib.Path("D:/ПРОГА/Проектики/Github/Electronic_library/dataset/Образование")
#books = pathlib.Path("D:/ПРОГА/Проектики/Github/Electronic_library/dataset/Спорт")
#books = pathlib.Path("D:/ПРОГА/Проектики/Github/Electronic_library/dataset/Космос")
#books = pathlib.Path("D:/ПРОГА/Проектики/Github/Electronic_library/dataset/Химия")
books = pathlib.Path("D:/ПРОГА/Проектики/Github/Electronic_library/dataset/Искусствоведение")
files_to_process = []
for book in books.iterdir():
    files_to_process.append(str(book))

In [167]:
files_to_process

['D:\\ПРОГА\\Проектики\\Github\\Electronic_library\\dataset\\Искусствоведение\\1.pdf',
 'D:\\ПРОГА\\Проектики\\Github\\Electronic_library\\dataset\\Искусствоведение\\10.pdf',
 'D:\\ПРОГА\\Проектики\\Github\\Electronic_library\\dataset\\Искусствоведение\\11.pdf',
 'D:\\ПРОГА\\Проектики\\Github\\Electronic_library\\dataset\\Искусствоведение\\12.pdf',
 'D:\\ПРОГА\\Проектики\\Github\\Electronic_library\\dataset\\Искусствоведение\\13.pdf',
 'D:\\ПРОГА\\Проектики\\Github\\Electronic_library\\dataset\\Искусствоведение\\14.pdf',
 'D:\\ПРОГА\\Проектики\\Github\\Electronic_library\\dataset\\Искусствоведение\\15.pdf',
 'D:\\ПРОГА\\Проектики\\Github\\Electronic_library\\dataset\\Искусствоведение\\16.pdf',
 'D:\\ПРОГА\\Проектики\\Github\\Electronic_library\\dataset\\Искусствоведение\\17.pdf',
 'D:\\ПРОГА\\Проектики\\Github\\Electronic_library\\dataset\\Искусствоведение\\18.pdf',
 'D:\\ПРОГА\\Проектики\\Github\\Electronic_library\\dataset\\Искусствоведение\\19.pdf',
 'D:\\ПРОГА\\Проектики\\Github\\E

In [103]:
with open(files_to_process[0], "rb") as f:
    reader = PyPDF2.PdfReader(f)
    text = [page.extract_text() for page in reader.pages]

In [174]:
def process_info(file):
    with open(file, "rb") as f:
        reader = PyPDF2.PdfReader(f)
        text = ''.join([page.extract_text() for page in reader.pages])
    # Служебная информация
    text = re.sub(r'ISSN\s+\d{4}-\d{3,4}[^\n]*', '', text)
    text = re.sub(r'\d{4};\d{2}\(\d+\):\d+–\d+', '', text)

    # Авторы
    text = re.sub(r'[А-ЯЁ][а-яё]+\s+[А-ЯЁ][\.\s]+\s*[А-ЯЁ][\.\s]*', '', text)
    text = re.sub(r'[А-ЯЁ][а-яё]+\s+[А-ЯЁ][а-яё]+\s+[А-ЯЁ][\.\s]+\s*[А-ЯЁ][\.\s]*', '', text)
    text = re.sub(r'\d+[\s\w\.,–-]+(университет|институт|академия|центр)[^\n]*', '', text)

    # Сноски в квадратных скобках
    text = re.sub(r'\[\d+\]', '', text)  # [1], [2]
    text = re.sub(r'\[\d+[,-]\d+\]', '', text)  # [1-3], [4,5]
    text = re.sub(r'\[[A-Za-z]+\d*\]', '', text)  # [A1], [B]
    
    # email
    text = re.sub(r'\S+@\S+', '', text)
    
    # английские разделы 
    text = re.sub(r'Abstract[^\n]*[\s\S]*?(?=\n[А-ЯЁ]|$)', '', text)
    text = re.sub(r'Keywords[^\n]*[\s\S]*?(?=\n[А-ЯЁ]|$)', '', text)
    text = re.sub(r'For citation[^\n]*[\s\S]*?(?=\n[А-ЯЁ]|$)', '', text)
    
    # ссылки
    text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text)
    text = re.sub(r'DOI:\s*\S+', '', text)
    
    # библиография
    text = re.sub(r'Список\s+источников[\s\S]*', '', text)
    text = re.sub(r'References[\s\S]*', '', text)
    
    # спец.символы
    text = text.replace('\xa0', ' ').replace('•', '')
    text = re.sub(r'-\s+', '', text)  
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'•\s*\n', '', text)
    
    # оставшиеся английские фрагменты
    text = re.sub(r'(?:[A-Za-z-]+\s){3,}[A-Za-z-]*', '', text)
    
    return text

## Тексты образования

In [175]:
books = pathlib.Path("D:/ПРОГА/Проектики/Github/Electronic_library/dataset/Образование")
files_to_process = []
for book in books.iterdir():
    files_to_process.append(str(book))

clean_files = []
for file in files_to_process:
    clean_files.append(process_info(file))

clean_data['Informatics'] = clean_files

## Тексты спорта

In [178]:
books = pathlib.Path("D:/ПРОГА/Проектики/Github/Electronic_library/dataset/Спорт")
files_to_process = []
for book in books.iterdir():
    files_to_process.append(str(book))

clean_files = []
for file in files_to_process:
    clean_files.append(process_info(file))

clean_data['Sport'] = clean_files

## Тексты космоса

In [179]:
books = pathlib.Path("D:/ПРОГА/Проектики/Github/Electronic_library/dataset/Космос")
files_to_process = []
for book in books.iterdir():
    files_to_process.append(str(book))

clean_files = []
for file in files_to_process:
    clean_files.append(process_info(file))

clean_data['Space'] = clean_files

## Тексты химии

In [180]:
books = pathlib.Path("D:/ПРОГА/Проектики/Github/Electronic_library/dataset/Химия")
files_to_process = []
for book in books.iterdir():
    files_to_process.append(str(book))

clean_files = []
for file in files_to_process:
    clean_files.append(process_info(file))

clean_data['Chemical'] = clean_files

## Тексты искусствоведения

In [181]:
books = pathlib.Path("D:/ПРОГА/Проектики/Github/Electronic_library/dataset/Искусствоведение")
files_to_process = []
for book in books.iterdir():
    files_to_process.append(str(book))

clean_files = []
for file in files_to_process:
    clean_files.append(process_info(file))

clean_data['Art'] = clean_files

In [188]:
clean_data.shape

(20, 5)

## Создание датасета

In [185]:
clean_data.to_csv('clean_data.csv', sep=',', index=False, encoding='utf-8')

In [186]:
data = pd.read_csv('clean_data.csv')

In [187]:
data.head()

,Informatics,Art,Space,Sport,Chemical
0,ПЕДАГОГИЧЕСКИЙ ОПЫТ / PEDAGOGICAL EXPERIENCE г...,8 РОССИЙСКИХ ЦЕННОСТЕЙ В КОНКРЕТНОИСТОРИЧЕСКОМ...,Солнечно -земная физика. 2024. Т. 10. № 4 Soln...,Ученые записки университета имени П.Ф. Лесгафт...,ХИМИЯ РАСТИТЕЛЬНОГО СЫРЬЯ . 2025 . №1. С. 106–...
1,"ИНФОРМАТИЗАЦИЯ ОБРАЗОВАНИЯ / г. Москва, Россия...",110 БАУХАУСА И ИХ ВЛИЯНИЕ НА ТЕОРИЮ АРХИТЕКТУР...,Солнечно -земная физика. 2025. Т. 11. № 1 Soln...,Ученые записки университета имени П.Ф. Лесгафт...,ХИМИЯ РАСТИТЕЛЬНОГО СЫРЬЯ . 2025. №1. С. 188–1...
2,ИНФОРМАТИЗАЦИЯ ОБРАЗОВАНИЯ / Аннотация В повыш...,117 ⇒ Художественная культура. ЭстетикаТ ЕАТРА...,Солнечно -земная физика. 2025. Т. 11. № 1 Soln...,Ученые записки университета имени П.Ф. Лесгафт...,ХИМИЯ РАСТИТЕЛЬНОГО СЫРЬЯ . 2025. №1. С. 197–2...
3,ИНФОРМАТИЗАЦИЯ ОБРАЗОВАНИЯ / Аннотация В стать...,"124 ДРАМАТУРГИИ ТЕЛЕСЕРИАЛА УДК 801 Химки, Мос...",Солнечно -земная физика. 2025. Т. 11. № 1 Soln...,Ученые записки университета имени П.Ф. Лесгафт...,ХИМИЯ РАСТИТЕЛЬНОГО СЫРЬЯ . 2025. №1. С. 208–2...
4,ПЕДАГОГИЧЕСКИЙ ОПЫТ / PEDAGOGICAL EXPERIENCE А...,132 В ТЕАТРАЛЬНЫХ ОПЫТАХ А. ВАСИЛЬЕВА УДК 740 ...,Солнечно -земная физика. 2025. Т. 11. № 1 Soln...,Ученые записки университета имени П.Ф. Лесгафт...,ХИМИЯ РАСТИТЕЛЬНОГО СЫРЬЯ . 2025. №1. С. 215–2...


In [190]:
#dcascdsa